In [1]:
import pandas as pd
import numpy as np
import datetime
import os

In [2]:
train_data_original = pd.read_csv("./data/train.csv.zip")
test_data = pd.read_csv("./data/test.csv.zip")

train_data = train_data_original.copy(deep=True)
data_cleaner = [train_data, test_data]

In [3]:
train_data.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [4]:
test_data.head()

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1


In [5]:
train_data.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [6]:
test_data.describe()

,id,store,item
count,45000.000000,45000.000000,45000.00000
mean,22499.500000,5.500000,25.50000
std,12990.525394,2.872313,14.43103
min,0.000000,1.000000,1.00000
25%,11249.750000,3.000000,13.00000
50%,22499.500000,5.500000,25.50000
75%,33749.250000,8.000000,38.00000
max,44999.000000,10.000000,50.00000


In [7]:
df = pd.concat([train_data, test_data], sort=False)

In [8]:
# get a list of us federal holidays as a datetime object
from pandas.tseries.holiday import USFederalHolidayCalendar
days_off = USFederalHolidayCalendar().holidays(start='2013-01-01', end='2017-12-27').to_pydatetime()

In [9]:
df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)
df['month'] = df['date'].dt.month
df['weekday'] = df['date'].dt.dayofweek
df['year'] = df['date'].dt.year
df['day'] = df['date'].dt.day
df['weekofyear']  = df.date.dt.weekofyear
df['dayofyear']  = df.date.dt.dayofyear
# # checks if holiday 
df['isHoliday'] = df['date'].isin(days_off).astype(int)

In [10]:
# more date features
# checks if day of week is saturday or sunday 
df['isWeekend'] = df['date'].map(lambda x: 1 if (x==6 or x==5) else 0)

# check for season 
print('checking for season')
# summer: check if month is between june and august (6-8)
df['isSummer'] = df['month'].map(lambda x : 1 if x == 6 or x == 7 or x == 8 else 0)
# winter: check if month is between dec and feb (12-2)
df['isWinter'] = df['month'].map(lambda x : 1 if x == 12 or x == 1 or x == 2 else 0)
# autumn: check if month is between sept and nov (9-11)
df['isAutumn'] = df['month'].map(lambda x : 1 if x == 9 or x == 10 or x == 11 else 0)
# spring: check if month is between march and may (3-5)
df['isSpring'] = df['month'].map(lambda x : 1 if x == 3 or x == 4 or x == 5 else 0)

checking for season


In [11]:
df[['month', 'isSummer', 'isAutumn', 'isWinter', 'isSpring']].tail()

,month,isSummer,isAutumn,isWinter,isSpring
44995,3,0,0,0,1
44996,3,0,0,0,1
44997,3,0,0,0,1
44998,3,0,0,0,1
44999,3,0,0,0,1


In [12]:
df.isnull().sum()

date               0
store              0
item               0
sales          45000
id            913000
month              0
weekday            0
year               0
day                0
weekofyear         0
dayofyear          0
isHoliday          0
isWeekend          0
isSummer           0
isWinter           0
isAutumn           0
isSpring           0
dtype: int64

In [13]:
print("year:", df['year'].unique())
print("month:", len(df['month'].unique()))
print("day:", len(df['day'].unique()))
print("weekofyear:", len(df['weekofyear'].unique()))
print("dayofyear:", len(df['dayofyear'].unique()))
print("weekday:", len(df['weekday'].unique()))
print("isWeekend:", df['isWeekend'].unique())
print("isHoliday:", df['isHoliday'].unique())
print("isSummer:", df['isSummer'].unique())
print("isAutumn:", df['isAutumn'].unique())
print("isSpring:", df['isSpring'].unique())
print("isWinter:", df['isSpring'].unique())

year: [2013 2014 2015 2016 2017 2018]
month: 12
day: 31
weekofyear: 53
dayofyear: 366
weekday: 7
isWeekend: [0]
isHoliday: [1 0]
isSummer: [0 1]
isAutumn: [0 1]
isSpring: [0 1]
isWinter: [0 1]


# Getting tranformation for datetime columns

In [14]:
datetime_cols = ['month', 'day', 'weekofyear', 'dayofyear', 'weekday', 'isWeekend', 'isHoliday', 'isSummer', 'isAutumn', 'isSummer', 'isWinter']
for col in datetime_cols:
    df[f'{col}_sum'] = df.groupby([f'{col}'])['sales'].transform('sum')
    df[f'{col}_median'] = df.groupby([f'{col}'])['sales'].transform('median')
    df[f'{col}_mean'] = df.groupby([f'{col}'])['sales'].transform('mean')
    print(f'done: {col}')

done: month
done: day
done: weekofyear
done: dayofyear
done: weekday
done: isWeekend
done: isHoliday
done: isSummer
done: isAutumn
done: isSummer
done: isWinter


In [15]:
df.isnull().sum()

date                      0
store                     0
item                      0
sales                 45000
id                   913000
month                     0
weekday                   0
year                      0
day                       0
weekofyear                0
dayofyear                 0
isHoliday                 0
isWeekend                 0
isSummer                  0
isWinter                  0
isAutumn                  0
isSpring                  0
month_sum                 0
month_median              0
month_mean                0
day_sum                   0
day_median                0
day_mean                  0
weekofyear_sum            0
weekofyear_median         0
weekofyear_mean           0
dayofyear_sum             0
dayofyear_median          0
dayofyear_mean            0
weekday_sum               0
weekday_median            0
weekday_mean              0
isWeekend_sum             0
isWeekend_median          0
isWeekend_mean            0
isHoliday_sum       

In [16]:
df[['dayofyear', 'dayofyear_median', 'weekday', 'weekday_median', 'day_median', 'isHoliday', 'isHoliday_median', 'isWeekend', 'isWeekend_median']].head()

,dayofyear,dayofyear_median,weekday,weekday_median,day_median,isHoliday,isHoliday_median,isWeekend,isWeekend_median
0,1,33.0,1,44.0,47.0,1,38.0,0,47.0
1,2,31.0,2,44.0,47.0,0,47.0,0,47.0
2,3,34.0,3,47.0,47.0,0,47.0,0,47.0
3,4,33.0,4,50.0,47.0,0,47.0,0,47.0
4,5,33.0,5,53.0,47.0,0,47.0,0,47.0


In [17]:
df.tail()

,date,store,item,sales,id,month,weekday,year,day,weekofyear,...,isHoliday_mean,isSummer_sum,isSummer_median,isSummer_mean,isAutumn_sum,isAutumn_median,isAutumn_mean,isWinter_sum,isWinter_median,isWinter_mean
44995,2018-03-27,10,50,NaN,44995.0,3,1,2018,27,13,...,52.508233,33204553.0,44.0,48.615744,35465179.0,46.0,51.73622,39124378.0,52.0,56.908186
44996,2018-03-28,10,50,NaN,44996.0,3,2,2018,28,13,...,52.508233,33204553.0,44.0,48.615744,35465179.0,46.0,51.73622,39124378.0,52.0,56.908186
44997,2018-03-29,10,50,NaN,44997.0,3,3,2018,29,13,...,52.508233,33204553.0,44.0,48.615744,35465179.0,46.0,51.73622,39124378.0,52.0,56.908186
44998,2018-03-30,10,50,NaN,44998.0,3,4,2018,30,13,...,52.508233,33204553.0,44.0,48.615744,35465179.0,46.0,51.73622,39124378.0,52.0,56.908186
44999,2018-03-31,10,50,NaN,44999.0,3,5,2018,31,13,...,52.508233,33204553.0,44.0,48.615744,35465179.0,46.0,51.73622,39124378.0,52.0,56.908186


# Getting transformation for store/Items Column

In [18]:
columns_to_sum_median_mean = ['store', 'item']

In [19]:
# store/item sales sum/median
for col in columns_to_sum_median_mean:
    df[f'{col}_sales_sum'] = df.groupby([f'{col}'])['sales'].transform('sum')
    df[f'{col}_sales_median'] = df.groupby([f'{col}'])['sales'].transform('median')
    df[f'{col}_sales_mean'] = df.groupby([f'{col}'])['sales'].transform('mean')

    # ---------------------------------------------------
    df[f'{col}_month_sales_sum'] = df.groupby(['month',f'{col}'])['sales'].transform('sum')
    df[f'{col}_month_sales_median'] = df.groupby(['month',f'{col}'])['sales'].transform('median')
    # store sales day sum/median 
    df[f'{col}_day_sales_sum'] = df.groupby(['day',f'{col}'])['sales'].transform('sum')
    df[f'{col}_day_sales_median'] = df.groupby(['day',f'{col}'])['sales'].transform('median')
    # store sales weekday sum/median
    df[f'{col}_weekday_sales_sum'] = df.groupby(['weekday',f'{col}'])['sales'].transform('sum')
    df[f'{col}_weekday_sales_median'] = df.groupby(['weekday',f'{col}'])['sales'].transform('median')
    # store dayofyear sum/median
    df[f'{col}_dayofyear_sales_sum'] = df.groupby(['dayofyear',f'{col}'])['sales'].transform('sum')
    df[f'{col}_dayofyear_sales_median'] = df.groupby(['dayofyear',f'{col}'])['sales'].transform('median')
    # store weekofyear sum/median 
    df[f'{col}_weekofyear_sales_sum'] = df.groupby(['weekofyear',f'{col}'])['sales'].transform('sum')
    df[f'{col}_weekofyear_sales_median'] = df.groupby(['weekofyear',f'{col}'])['sales'].transform('median')

    # --------------------------------------------------- 
    # final one 
    # store sales year/month/day/weekday/dayofyear/weekofyear/ sum/median 
    
    print(f'finished {col}')

finished store
finished item


In [20]:
df.isnull().sum()

date                                  0
store                                 0
item                                  0
sales                             45000
id                               913000
month                                 0
weekday                               0
year                                  0
day                                   0
weekofyear                            0
dayofyear                             0
isHoliday                             0
isWeekend                             0
isSummer                              0
isWinter                              0
isAutumn                              0
isSpring                              0
month_sum                             0
month_median                          0
month_mean                            0
day_sum                               0
day_median                            0
day_mean                              0
weekofyear_sum                        0
weekofyear_median                     0


In [21]:
df.tail()

,date,store,item,sales,id,month,weekday,year,day,weekofyear,...,item_month_sales_sum,item_month_sales_median,item_day_sales_sum,item_day_sales_median,item_weekday_sales_sum,item_weekday_sales_median,item_dayofyear_sales_sum,item_dayofyear_sales_median,item_weekofyear_sales_sum,item_weekofyear_sales_median
44995,2018-03-27,10,50,NaN,44995.0,3,1,2018,27,13,...,92609.0,59.0,39807.0,64.0,158226.0,59.0,2955.0,59.5,21496.0,59.0
44996,2018-03-28,10,50,NaN,44996.0,3,2,2018,28,13,...,92609.0,59.0,39254.0,62.0,159695.0,59.0,3088.0,60.5,21496.0,59.0
44997,2018-03-29,10,50,NaN,44997.0,3,3,2018,29,13,...,92609.0,59.0,37613.0,66.0,171078.0,63.0,2959.0,57.5,21496.0,59.0
44998,2018-03-30,10,50,NaN,44998.0,3,4,2018,30,13,...,92609.0,59.0,37258.0,66.0,181066.0,67.0,2993.0,58.5,21496.0,59.0
44999,2018-03-31,10,50,NaN,44999.0,3,5,2018,31,13,...,92609.0,59.0,22745.0,62.0,192856.0,71.0,2851.0,58.5,21496.0,59.0


# Getting Transformations For Store_Item Columns

In [22]:
# store sales sum/median
df['store_item_sales_sum'] = df.groupby(['store','item'])['sales'].transform('sum')
df['store_item_sales_median'] = df.groupby(['store','item'])['sales'].transform('median')
df['store_item_sales_mean'] = df.groupby(['store','item'])['sales'].transform('mean')

# ---------------------------------------------------
# store sales month sum/median 
df['store_item_month_sales_sum'] = df.groupby(['month','store','item'])['sales'].transform('sum')
df['item_month_sales_median'] = df.groupby(['month','store','item'])['sales'].transform('median')
# store sales day sum/median 
df['store_item_day_sales_sum'] = df.groupby(['day','store','item'])['sales'].transform('sum')
df['store_item_day_sales_median'] = df.groupby(['day','store','item'])['sales'].transform('median')
# store sales weekday sum/median
df['store_item_weekday_sales_sum'] = df.groupby(['weekday','store','item'])['sales'].transform('sum')
df['store_item_weekday_sales_median'] = df.groupby(['weekday','store','item'])['sales'].transform('median')
# store dayofyear sum/median
df['store_item_dayofyear_sales_sum'] = df.groupby(['dayofyear','store','item'])['sales'].transform('sum')
df['store_item_dayofyear_sales_median'] = df.groupby(['dayofyear','store','item'])['sales'].transform('median')
# store weekofyear sum/median 
df['store_item_weekofyear_sales_sum'] = df.groupby(['weekofyear','store','item'])['sales'].transform('sum')
df['store_item_weekofyear_sales_median'] = df.groupby(['weekofyear','store','item'])['sales'].transform('median')

In [23]:
df.isnull().sum()

date                                       0
store                                      0
item                                       0
sales                                  45000
id                                    913000
month                                      0
weekday                                    0
year                                       0
day                                        0
weekofyear                                 0
dayofyear                                  0
isHoliday                                  0
isWeekend                                  0
isSummer                                   0
isWinter                                   0
isAutumn                                   0
isSpring                                   0
month_sum                                  0
month_median                               0
month_mean                                 0
day_sum                                    0
day_median                                 0
day_mean  

In [24]:
df.tail()

,date,store,item,sales,id,month,weekday,year,day,weekofyear,...,store_item_sales_mean,store_item_month_sales_sum,store_item_day_sales_sum,store_item_day_sales_median,store_item_weekday_sales_sum,store_item_weekday_sales_median,store_item_dayofyear_sales_sum,store_item_dayofyear_sales_median,store_item_weekofyear_sales_sum,store_item_weekofyear_sales_median
44995,2018-03-27,10,50,NaN,44995.0,3,1,2018,27,13,...,74.03724,10506.0,4390.0,69.5,17690.0,68.0,345.0,67.0,2407.0,68.0
44996,2018-03-28,10,50,NaN,44996.0,3,2,2018,28,13,...,74.03724,10506.0,4507.0,73.0,17963.0,71.0,338.0,60.0,2407.0,68.0
44997,2018-03-29,10,50,NaN,44997.0,3,3,2018,29,13,...,74.03724,10506.0,4206.0,73.5,19192.0,73.0,341.0,73.0,2407.0,68.0
44998,2018-03-30,10,50,NaN,44998.0,3,4,2018,30,13,...,74.03724,10506.0,4112.0,74.0,20430.0,79.0,344.0,68.0,2407.0,68.0
44999,2018-03-31,10,50,NaN,44999.0,3,5,2018,31,13,...,74.03724,10506.0,2583.0,73.0,21625.0,83.0,324.0,67.0,2407.0,68.0


# Export processed data

In [26]:
# train_data.to_csv('./data/preprocessed_train_data.csv')
# test_data.to_csv('./data/preprocessed_test_data.csv')
df.to_csv('./data/preprocessed_train_test_data.csv', index=False)